Partial credits for this code goes to : https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/

In [38]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from models.LeNet import LeNet
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time

#### Hyperparameters

In [39]:
# Define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10

# Define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT

# Configure the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Argument parser

In [40]:
# Construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-m", "--model", type=str, required=True, help="path to output trained model")
#ap.add_argument("-p", "--plot", type=str, required=True, help="path to output loss/accuracy plot")
#args = vars(ap.parse_args())

#### Loading dataset

In [41]:
print("[INFO] loading dataset...")
trainData = MNIST(root="../data", train=True, download=True, transform=ToTensor())
testData = MNIST(root="../data", train=False, download=True, transform=ToTensor())

# Calculate the train/validation split
print("[INFO] generating the train/validation split...")
numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = int(len(trainData) * VAL_SPLIT)
(trainData, valData) = random_split(trainData, [numTrainSamples, numValSamples], generator=torch.Generator().manual_seed(42))

[INFO] loading dataset...
[INFO] generating the train/validation split...


In [42]:
# Initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(trainData, shuffle=True, batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)

# Calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [43]:
# Initialize the model
print("[INFO] initializing the LeNet model...")
model = LeNet(numChannels=1, classes=len(trainData.dataset.classes)).to(device)

# Initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()

[INFO] initializing the LeNet model...


In [47]:
# loop over our epochs
for e in range(0, EPOCHS):
	# Set the model in training mode
	model.train()
	# Initialize the total training/validation loss/correct predictions
	totalTrainLoss, totalValLoss, trainCorrect, valCorrect = 0, 0, 0, 0
	
	# Training
	for (x, y) in trainDataLoader:
		# Send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# Perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# Zero out the gradients, perform the backpropagation step, and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# Sum the loss to the total training loss so far
		totalTrainLoss += loss
		# Calculate the number of correct predictions
		trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()
	
	# Validation
	with torch.no_grad():
		# Set the model in evaluation mode
		model.eval()
		# Loop over the validation set
		for (x, y) in valDataLoader:
			# Send the input to the device
			(x, y) = (x.to(device), y.to(device))
			# Make the predictions and calculate the validation loss
			pred = model(x)
			totalValLoss += lossFn(pred, y)
			# Calculate the number of correct predictions
			valCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

	# Calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# Calculate the training and validation accuracy
	trainCorrect = trainCorrect / len(trainDataLoader.dataset)
	valCorrect = valCorrect / len(valDataLoader.dataset)

	# Update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
	# Print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(avgValLoss, valCorrect))
	
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1568 and 800x500)

In [ ]:
# We can now evaluate the network on the test set
print("[INFO] evaluating network...")

with torch.no_grad(): # Turn off autograd for testing evaluation
	model.eval() # Set the model in evaluation mode
	preds = [] # Initialize a list to store our predictions
	# Evaluation with test dataset
	for (x, y) in testDataLoader:
		# Send the input to the device
		x = x.to(device)
		# Make the predictions and add them to the list
		pred = model(x)
		preds.extend(pred.argmax(axis=1).cpu().numpy())
		
# Generate a classification report
print(classification_report(testData.targets.cpu().numpy(), np.array(preds), target_names=testData.classes))

In [ ]:
# Plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

# Serialize the model to disk
torch.save(model, args["model"])